# Lazada Web Scraping

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import bs4
import pandas as pd
import re

### Service and Driver

In [2]:
service = Service(r'C:\Users\User\Desktop\ETLPY\Sentiment_Analysis\nlp\Web_Scraping\chromedriver\chromedriver.exe')
driver = webdriver.Chrome(service=service)

### Url Lazada

In [3]:
driver.get('https://www.lazada.co.th/')

### Find Search bar

In [4]:
search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div[2]/div/form/div/div[1]/input[1]')

In [5]:
search.send_keys('ชีสทาร์ตสตอเบอรี่')

In [6]:
search.send_keys(Keys.ENTER)

### Find Content Box

In [7]:
Content_Box = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/div[1]/div/div[1]/div[2]/div[1]')

In [8]:
Content_Box.click()

### Zoom

In [9]:
driver.execute_script("document.body.style.zoom='20%'")

### Page Source Data

In [10]:
data = driver.page_source

In [11]:
soup = bs4.BeautifulSoup(data)

### Content data

all_content = soup.find_all('div',{'class':'content'})

all_content_list = []
for content in all_content:
    all_content_list.append(content.text)
    
df_content = pd.DataFrame(all_content_list)
print(df_content)

for content in all_content:
    print(content.text)
    print()

### Star data

rating_containers = driver.find_elements(By.CSS_SELECTOR, "div.container-star.starCtn.left")

ratings = []

# Iterate through each container to count stars
for container in rating_containers:
    try:
        stars = container.find_elements(By.CSS_SELECTOR, "img.star")  # Locate stars within the container
        if stars:
            ratings.append(len(stars))  # Count the stars and append the count
        else:
            ratings.append("none")  # Add "none" if no stars are found
    except Exception as e:
        ratings.append(f"error: {e}")  # Handle any potential errors

# Display the ratings
for index, rating in enumerate(ratings, start=1):
    print(f"รายการที่ {index}: {rating} ดาว")

### Loop Test

import re

# เก็บข้อมูลทั้งหมด
all_content_list = []
ratings = []

# เริ่มต้น While Loop
while True:
    try:
        # ดึงข้อมูลเนื้อหาจาก div ที่มีคลาส 'content'
        all_content = soup.find_all('div', {'class': 'content'})
        temp_content = []  # ใช้เก็บข้อมูล content ในรอบนี้
        for content in all_content:
            text = content.text.strip()  # ตัดช่องว่างก่อนและหลังข้อความ
            # ตรวจสอบข้อความที่ไม่ต้องการ
            if not re.search(r"^\d+\.\d+/\d+ Ratings\d+$", text):
                temp_content.append(text)  # เก็บข้อความที่ไม่ตรงกับรูปแบบ
        
        # เก็บข้อมูล Rating จาก container-star
        rating_containers = driver.find_elements(By.CSS_SELECTOR, "div.container-star.starCtn.left")
        temp_ratings = []  # ใช้เก็บข้อมูล ratings ในรอบนี้
        for container in rating_containers:
            try:
                stars = container.find_elements(By.CSS_SELECTOR, "img.star")
                if stars:
                    temp_ratings.append(len(stars))  # นับจำนวนดาว
                else:
                    temp_ratings.append("none")  # กรณีไม่มีดาว
            except Exception as e:
                temp_ratings.append(f"error: {e}")  # หากมีข้อผิดพลาด

        # ตรวจสอบว่าข้อมูล Content และ Ratings มีจำนวนเท่ากัน
        if len(temp_content) != len(temp_ratings):
            print("จำนวน Content และ Ratings ไม่ตรงกัน เติมค่า 'none'")
            # เติมค่า "none" ให้ข้อมูลที่ขาดหาย
            while len(temp_content) > len(temp_ratings):
                temp_ratings.append("none")
            while len(temp_ratings) > len(temp_content):
                temp_content.append("none")

        # เพิ่มข้อมูลเข้าในรายการหลัก
        all_content_list.extend(temp_content)
        ratings.extend(temp_ratings)

        # สร้าง DataFrame จากข้อมูล
        df_content = pd.DataFrame({'Content': all_content_list, 'Rating': ratings})
        print(df_content)
        
        # คลิกปุ่ม Next
        nextButton = driver.find_element(By.XPATH, '/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')
        nextButton.click()

         # ดึงข้อมูล HTML ปัจจุบันจาก Selenium
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)

        # รอให้หน้าโหลดข้อมูลใหม่ก่อนเริ่ม Loop ใหม่
        time.sleep(5)  # รอ 5 วินาที (หรือปรับเวลาตามความเหมาะสม)

    except Exception as e:
        print("เกิดข้อผิดพลาด:", e)
        break  # หยุด Loop หากเกิดข้อผิดพลาด เช่น ไม่มีปุ่มถัดไป

# # บันทึกลงไฟล์ Excel
# df_content.to_excel('content_ratings_filtered.xlsx', index=False)
# print("บันทึกข้อมูลเรียบร้อยใน content_ratings_filtered.xlsx")


df_content = pd.DataFrame(all_content_list, columns=['Content'])

df_content.to_excel(r'C:\Users\User\Desktop\ETLPY\Sentiment_Analysis\nlp\Web_Scraping\lazada_data_case3.xlsx', index=False)

In [14]:
import re

# เก็บข้อมูลทั้งหมด
all_content_list = []
ratings = []
all_name_list = []
date_list = []
like_list = []

# เริ่มต้น While Loop
while True:
    try:
        # ดึงข้อมูลเนื้อหาจาก div ที่มีคลาส 'content'
        all_content = soup.find_all('div', {'class': 'content'})
        temp_content = []  # ใช้เก็บข้อมูล content ในรอบนี้
        for content in all_content:
            text = content.text.strip()  # ตัดช่องว่างก่อนและหลังข้อความ
            # ตรวจสอบข้อความที่ไม่ต้องการ
            if not re.search(r"^\d+\.\d+/\d+ Ratings\d+$", text):
                temp_content.append(text)  # เก็บข้อความที่ไม่ตรงกับรูปแบบ
        
        # ดึงข้อมูล Rating จาก container-star
        rating_containers = driver.find_elements(By.CSS_SELECTOR, "div.container-star.starCtn.left")
        temp_ratings = []  # ใช้เก็บข้อมูล ratings ในรอบนี้
        for container in rating_containers:
            try:
                stars = container.find_elements(By.CSS_SELECTOR, "img.star")
                if stars:
                    temp_ratings.append(len(stars))  # นับจำนวนดาว
                else:
                    temp_ratings.append("none")  # กรณีไม่มีดาว
            except Exception as e:
                temp_ratings.append(f"error: {e}")  # หากมีข้อผิดพลาด

        # ดึงข้อมูลชื่อจาก middle
        all_name = soup.find_all('div', {'class': 'middle'})
        temp_names = []
        for name in all_name:
            span = name.find('span')  # ค้นหาแท็ก span ภายใน div ที่เลือกไว้
            if span:
                temp_names.append(span.text.strip())  # ดึงข้อความและตัดช่องว่าง

        # ดึงข้อมูลวันที่จาก title right
        all_date = soup.find_all('span', {'class': 'title right'})
        temp_dates = []
        for date in all_date:
            temp_dates.append(date.text.strip())  # ดึงข้อความและลบช่องว่าง

        # ดึงข้อมูล like จาก left-content
        all_like = soup.find_all('span', {'class': 'left-content'})
        temp_likes = []
        for like in all_like:
            span = like.find('span')  # ค้นหาแท็ก span ที่อยู่ภายใน
            if span:
                temp_likes.append(span.text.strip())  # ดึงข้อความและตัดช่องว่าง

        # ตรวจสอบว่าข้อมูล Content และ Ratings มีจำนวนเท่ากัน
        if len(temp_content) != len(temp_ratings):
            print("จำนวน Content และ Ratings ไม่ตรงกัน เติมค่า 'none'")
            # เติมค่า "none" ให้ข้อมูลที่ขาดหาย
            while len(temp_content) > len(temp_ratings):
                temp_ratings.append("none")
            while len(temp_ratings) > len(temp_content):
                temp_content.append("none")

        # เพิ่มข้อมูลเข้าในรายการหลัก
        all_content_list.extend(temp_content)
        ratings.extend(temp_ratings)
        all_name_list.extend(temp_names)
        date_list.extend(temp_dates)
        like_list.extend(temp_likes)

        # สร้าง DataFrame จากข้อมูลทั้งหมด
        df_content = pd.DataFrame({
            'Content': all_content_list,
            'Rating': ratings,
            'Name': all_name_list,
            'Date': date_list,
            'Like': like_list
        })
        print(df_content)
        
        # คลิกปุ่ม Next
        nextButton = driver.find_element(By.XPATH, '/html/body/div[5]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')
        nextButton.click()

        # ดึงข้อมูล HTML ปัจจุบันจาก Selenium
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)

        # รอให้หน้าโหลดข้อมูลใหม่ก่อนเริ่ม Loop ใหม่
        time.sleep(5)  # รอ 5 วินาที (หรือปรับเวลาตามความเหมาะสม)

    except Exception as e:
        print("เกิดข้อผิดพลาด:", e)
        break  # หยุด Loop หากเกิดข้อผิดพลาด เช่น ไม่มีปุ่มถัดไป

# # บันทึกลงไฟล์ Excel
# df_content.to_excel('content_ratings_filtered.xlsx', index=False)
# print("บันทึกข้อมูลเรียบร้อยใน content_ratings_filtered.xlsx")


                                             Content  Rating         Name  \
0  แพ็กเกจจิ้งสวยเหมือนขนมจากญี่ปุ่นเลย เสียดายมา...       5        i***.   
1  ทางร้านแพ็คสินค้ามาเป็นอย่างดีเหมือนเดิม ชีสทา...       5      Ninz C.   
2  สั่งรอบ2แล้ว ขนมรสชาดดี บราวนี่อร่อย รอบนี้ใช้...       5    เกศมณี ว.   
3  สินค้าคุณภาพดี ราคาไม่แพง จัดส่งรวดเร็ว ประทับ...       5        W***.   
4  คุณค่าของเงิน: รสชาติ:ได้รับสินค้าเรียบร้อยแล้...       5  แสงสว่าง ใ.   

          Date Like  
0  10 Mar 2023    0  
1  10 Mar 2023    1  
2  09 Mar 2023    0  
3  08 Mar 2023    0  
4  06 Mar 2023    0  
                                             Content  Rating         Name  \
0  แพ็กเกจจิ้งสวยเหมือนขนมจากญี่ปุ่นเลย เสียดายมา...       5        i***.   
1  ทางร้านแพ็คสินค้ามาเป็นอย่างดีเหมือนเดิม ชีสทา...       5      Ninz C.   
2  สั่งรอบ2แล้ว ขนมรสชาดดี บราวนี่อร่อย รอบนี้ใช้...       5    เกศมณี ว.   
3  สินค้าคุณภาพดี ราคาไม่แพง จัดส่งรวดเร็ว ประทับ...       5        W***.   
4  คุณค่าของเงิน: รส

KeyboardInterrupt: 

In [17]:
 # บันทึกลงไฟล์ Excel
df_content.to_excel('content_ratings_filtered2.xlsx', index=False)
print("บันทึกข้อมูลเรียบร้อยใน content_ratings_filtered.xlsx")

บันทึกข้อมูลเรียบร้อยใน content_ratings_filtered.xlsx
